**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Install Keycloak

Following https://www.keycloak.org/getting-started/getting-started-kube

and https://beyondthekube.com/identity-management-part-3-setting-up-oidc-authentication/

We are using the new `networking.k8s.io/v1` Ingress apiVersion here, this assumes minikube v1.23.x  

Create valid certificates for the ingress with [`mkcert`](MKCert.ipynb) first

In [136]:
CAROOT=$(mkcert -CAROOT)

echo $CAROOT


/home/thomas/.local/share/mkcert
rootCA-key.pem  rootCA.pem


In [137]:
ls $CAROOT

rootCA-key.pem  rootCA.pem


In [141]:
mkdir -p $HOME/.minikube/certs


In [142]:
cp $CAROOT/rootCA.pem $HOME/.minikube/certs/mkcert.pem

In [143]:
ls  $HOME/.minikube/certs/

mkcert.pem


In [144]:
minikube start --cpus=8 --memory=8192 \
--extra-config=apiserver.oidc-client-id=kubernetes \
--extra-config=apiserver.oidc-username-claim=email \
--extra-config=apiserver.oidc-username-prefix=oidc: \
--extra-config=apiserver.oidc-groups-claim=groups \
--extra-config=apiserver.oidc-groups-prefix=oidc: \
--extra-config=apiserver.oidc-issuer-url=https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm

😄  minikube v1.23.2 auf Ubuntu 20.04
🎉  minikube 1.24.0 is available! Download it: https://github.com/kubernetes/minikube/releases/tag/v1.24.0
💡  To disable this notice, run: 'minikube config set WantUpdateNotification false'

✨  Automatically selected the docker driver. Other choices: kvm2, podman, none, ssh
❗  Your cgroup does not allow setting memory.
    ▪ More information: https://docs.docker.com/engine/install/linux-postinstall/#your-kernel-does-not-support-cgroup-swap-limit-capabilities
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=8, Memory=8192MB) ...
🐳  Vorbereiten von Kubernetes v1.22.2 auf Docker 20.10.8...
    ▪ apiserver.oidc-client-id=kubernetes
    ▪ apiserver.oidc-username-claim=email
    ▪ apiserver.oidc-username-prefix=oidc:
    ▪ apiserver.oidc-groups-claim=groups
    ▪ apiserver.oidc-groups-prefix=oidc:
    ▪ apiserver.oidc-issuer-url=https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm


In [129]:
mkdir rootca

In [61]:
cp rootca/rootCA.crt ~/.minikube/certs/nipio.pem

In [182]:
minikube stop
minikube start --embed-certs

✋  Stopping node "minikube"  ...
🛑  1 nodes stopped.
😄  minikube v1.23.2 auf Ubuntu 20.04
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔄  Restarting existing docker container for "minikube" ...
🐳  Vorbereiten von Kubernetes v1.22.2 auf Docker 20.10.8...
🔎  Verifying Kubernetes components...
    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
🔎  Verifying ingress addon...
🌟  Enabled addons: storage-provisioner, default-storageclass, ingress
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


In [145]:
minikube version

minikube version: v1.23.2
commit: 0a0ad764652082477c00d51d2475284b5d39ceed


In [146]:
minikube addons list

|-----------------------------|----------|--------------|-----------------------|
|         ADDON NAME          | PROFILE  |    STATUS    |      MAINTAINER       |
|-----------------------------|----------|--------------|-----------------------|
| ambassador                  | minikube | disabled     | unknown (third-party) |
| auto-pause                  | minikube | disabled     | google                |
| csi-hostpath-driver         | minikube | disabled     | kubernetes            |
| dashboard                   | minikube | disabled     | kubernetes            |
| default-storageclass        | minikube | enabled ✅   | kubernetes            |
| efk                         | minikube | disabled     | unknown (third-party) |
| freshpod                    | minikube | disabled     | google                |
| gcp-auth                    | minikube | disabled     | google                |
| gvisor                      | minikube | disabled     | google                |
| helm-tiller    

In [147]:
minikube addons enable ingress

    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


In [148]:
minikube addons list

|-----------------------------|----------|--------------|-----------------------|
|         ADDON NAME          | PROFILE  |    STATUS    |      MAINTAINER       |
|-----------------------------|----------|--------------|-----------------------|
| ambassador                  | minikube | disabled     | unknown (third-party) |
| auto-pause                  | minikube | disabled     | google                |
| csi-hostpath-driver         | minikube | disabled     | kubernetes            |
| dashboard                   | minikube | disabled     | kubernetes            |
| default-storageclass        | minikube | enabled ✅   | kubernetes            |
| efk                         | minikube | disabled     | unknown (third-party) |
| freshpod                    | minikube | disabled     | google                |
| gcp-auth                    | minikube | disabled     | google                |
| gvisor                      | minikube | disabled     | google                |
| helm-tiller    

In [172]:
kubectl create namespace keycloak

namespace/keycloak created


In [186]:
kubectl config set-context --current --namespace=keycloak

Context "minikube" modified.


In [150]:
ls certs

_wildcard.nip.io-key.pem  _wildcard.nip.io.pem


In [152]:
kubectl create secret tls nipio-tls -n kube-system \
  --cert=certs/_wildcard.nip.io.pem \
  --key=certs/_wildcard.nip.io-key.pem

secret/nipio-tls created


In [153]:
echo kube-system/nipio-tls | minikube addons configure ingress

-- Enter custom cert(format is "namespace/secret"): ✅  ingress was successfully configured


In [154]:
minikube addons disable ingress
minikube addons enable ingress

    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
🌑  "The 'ingress' addon is disabled
    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


In [166]:
export INGRESS_NGINX_CONTROLLER=$(kubectl get pods -n ingress-nginx -l app.kubernetes.io/component=controller -o jsonpath='{.items[0].metadata.name}')
echo $INGRESS_NGINX_CONTROLLER

ingress-nginx-controller-5898cbf9cf-zcdrh


In [169]:
kubectl exec -n ingress-nginx  $INGRESS_NGINX_CONTROLLER -- grep pem /etc/nginx/nginx.conf

	ssl_certificate     /etc/ingress-controller/ssl/kube-system-nipio-tls.pem;
	ssl_certificate_key /etc/ingress-controller/ssl/kube-system-nipio-tls.pem;


In [170]:
kubectl exec -n ingress-nginx  $INGRESS_NGINX_CONTROLLER -- \
   openssl x509 -in /etc/ingress-controller/ssl/kube-system-nipio-tls.pem -noout -text

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            6c:27:2b:b4:c2:ec:f1:ce:97:23:70:6d:aa:c5:02:2d
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: O = mkcert development CA, OU = thomas@hermione, CN = mkcert thomas@hermione
        Validity
            Not Before: Nov 14 15:45:25 2021 GMT
            Not After : Feb 14 15:45:25 2024 GMT
        Subject: O = mkcert development certificate, OU = thomas@hermione
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:cd:2b:82:c2:ea:eb:79:e6:40:19:17:9c:e3:69:
                    fe:3d:d7:9f:a2:da:38:ab:74:2a:92:c1:80:76:af:
                    39:99:8c:fa:fd:f0:ad:79:8b:d7:9a:be:01:06:02:
                    47:41:01:9f:af:ca:ef:d1:52:41:6e:c0:97:4f:b3:
                    95:f2:ce:fb:e6:d9:b4:8d:87:9b:ef:a8:2e:bb:71:
                    15:03:c6:07:11:00:87:a0:1b:75:6f:fe

## Install keycloak

In [173]:
curl -q https://raw.githubusercontent.com/keycloak/keycloak-quickstarts/latest/kubernetes-examples/keycloak.yaml | \
   sed s/default/keycloak/ | kubectl create -f -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   962  100   962    0     0   3073      0 --:--:-- --:--:-- --:--:--  3073
service/keycloak created
deployment.apps/keycloak created


In [185]:
kubectl get all

NAME                            READY   STATUS    RESTARTS        AGE
pod/keycloak-6655d58b6c-cqprk   1/1     Running   1 (3m20s ago)   75m

NAME               TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)          AGE
service/keycloak   LoadBalancer   10.106.150.139   10.106.150.139   8080:31636/TCP   75m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/keycloak   1/1     1            1           75m

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/keycloak-6655d58b6c   1         1         1       75m


run in a separate window

```
minikube tunnel
Status:	
	machine: minikube
	pid: 554606
	route: 10.96.0.0/12 -> 192.168.49.2
	minikube: Running
	services: [keycloak]
    errors: 
		minikube: no errors
		router: no errors
		loadbalancer emulator: no errors
...
```

In [187]:
KEYCLOAK_HOST=keycloak-$(minikube ip | sed s/\\./-/g).nip.io
echo $KEYCLOAK_HOST

keycloak-192-168-49-2.nip.io


Ingress version

https://kubernetes.io/docs/concepts/services-networking/ingress/

In [176]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1 ## new in 1.22
kind: Ingress
metadata:
  name: keycloak
  namespace: keycloak
spec:
  tls:
    - hosts:
      - $KEYCLOAK_HOST
      secretName: keycloak-tls
  rules:
  - host: $KEYCLOAK_HOST
    http:
      paths:
      - pathType: Prefix
        path: "/"
        backend:
          service:
            name: keycloak
            port: 
              number: 8080
EOF

ingress.networking.k8s.io/keycloak created


In [178]:
kubectl get ingress keycloak -o yaml

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"networking.k8s.io/v1","kind":"Ingress","metadata":{"annotations":{},"name":"keycloak","namespace":"keycloak"},"spec":{"rules":[{"host":"keycloak-192-168-49-2.nip.io","http":{"paths":[{"backend":{"service":{"name":"keycloak","port":{"number":8080}}},"path":"/","pathType":"Prefix"}]}}],"tls":[{"hosts":["keycloak-192-168-49-2.nip.io"],"secretName":"keycloak-tls"}]}}
  creationTimestamp: "2021-11-14T16:41:18Z"
  generation: 1
  name: keycloak
  namespace: keycloak
  resourceVersion: "1555"
  uid: 94825b4c-db01-4d07-a7b8-39439607038e
spec:
  ingressClassName: nginx
  rules:
  - host: keycloak-192-168-49-2.nip.io
    http:
      paths:
      - backend:
          service:
            name: keycloak
            port:
              number: 8080
        path: /
        pathType: Prefix
  tls:
  - hosts:
    - keycloak-192-168-49-2.nip.io
    secretN

In [179]:
KEYCLOAK_URL=https://$KEYCLOAK_HOST/auth &&
echo "" &&
echo "Keycloak:                 $KEYCLOAK_URL" &&
echo "Keycloak Admin Console:   $KEYCLOAK_URL/admin" &&
echo "Keycloak Account Console: $KEYCLOAK_URL/realms/myrealm/account" &&
echo ""


Keycloak:                 https://keycloak-192-168-49-2.nip.io/auth
Keycloak Admin Console:   https://keycloak-192-168-49-2.nip.io/auth/admin
Keycloak Account Console: https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/account



In [180]:
kubectl logs -l app=keycloak

16:41:20,321 INFO  [org.jboss.resteasy.resteasy_jaxrs.i18n] (ServerService Thread Pool -- 60) RESTEASY002220: Adding singleton resource org.keycloak.services.resources.WelcomeResource from Application class org.keycloak.services.resources.KeycloakApplication
16:41:20,321 INFO  [org.jboss.resteasy.resteasy_jaxrs.i18n] (ServerService Thread Pool -- 60) RESTEASY002210: Adding provider singleton org.keycloak.services.util.ObjectMapperResolver from Application class org.keycloak.services.resources.KeycloakApplication
16:41:20,321 INFO  [org.jboss.resteasy.resteasy_jaxrs.i18n] (ServerService Thread Pool -- 60) RESTEASY002220: Adding singleton resource org.keycloak.services.resources.RobotsResource from Application class org.keycloak.services.resources.KeycloakApplication
16:41:20,321 INFO  [org.jboss.resteasy.resteasy_jaxrs.i18n] (ServerService Thread Pool -- 60) RESTEASY002220: Adding singleton resource org.keycloak.services.resources.RealmsResource from Application class org.keycloak.servi

### Keycloak Realm

We need to create a Keycloak realm and user following

https://www.keycloak.org/getting-started/getting-started-kube

We assume to use `myrealm` and `myuser` for the job

![realm.png](realm.png)

### Keycloak User

Note after creating a the user `myuser` in the `myrealm` login to the user in the correct realm

https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/account

### Keycloak Client

Finally a client needs to be created for the specific usecase identified by the client-id `kubernetes`.

The redirect is relative to the browser, `kubelogin` 
- spawns a server on `localhost` port 8000
- inserts the token into `.kube/config`
- and terminates


![kubernetes-client.png](kubernetes-client.png)


In [181]:
curl -ks https://$KEYCLOAK_HOST/auth/realms/myrealm/.well-known/openid-configuration | jq

{
  "issuer": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm",
  "authorization_endpoint": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/auth",
  "token_endpoint": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/token",
  "introspection_endpoint": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/token/introspect",
  "userinfo_endpoint": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/userinfo",
  "end_session_endpoint": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/logout",
  "jwks_uri": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/certs",
  "check_session_iframe": "https://keycloak-192-168-49-2.nip.io/auth/realms/myrealm/protocol/openid-connect/login-status-iframe.html",
  "grant_types_supported": [
    "authorization_code",
    "implicit",
    "refresh_token",
    "p

In [84]:
docker exec minikube ls /etc/kubernetes/manifests

etcd.yaml
kube-apiserver.yaml
kube-controller-manager.yaml
kube-scheduler.yaml


In [85]:
docker exec minikube cat /etc/kubernetes/manifests/kube-apiserver.yaml | grep -E '(.*oidc.*|^)'

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubeadm.kubernetes.io/kube-apiserver.advertise-address.endpoint: 192.168.49.2:8443
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.49.2
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
    - --kubelet-clien

In [86]:
docker exec  minikube systemctl status

● minikube
    State: running
     Jobs: 0 queued
   Failed: 0 units
    Since: Fri 2021-11-12 17:24:35 UTC; 10min ago
   CGroup: /docker/e71fff49dee2a0695d134dc1fd8f047681661c5729181947458bbdfc21d9ae20
           ├─init.scope 
           │ └─1 /sbin/init
           └─system.slice 
             ├─containerd.service …
             │ ├─ 187 /usr/bin/containerd
             │ ├─1490 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 40887133ef44e81ab08bd3b8d1c61fc55eb3fcd12c9ec2716c93f0f41921a3d4 -address /run/containerd/containerd.sock
             │ ├─1492 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 062aaecda655a0da68b60e7fb729bb2dbbabb340aa30b6ace35ca890465ec63a -address /run/containerd/containerd.sock
             │ ├─1511 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 99aa4839bcf83d07752f3d8d07493c3426355f657a811f3a61a2f27f9b74c7e8 -address /run/containerd/containerd.sock
             │ ├─1536 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 9e0882019c11930283997

In [87]:
docker exec  minikube docker ps

CONTAINER ID   IMAGE                                 COMMAND                  CREATED          STATUS          PORTS                                      NAMES
bf110e33c2dd   quay.io/keycloak/keycloak             "/opt/jboss/tools/do…"   9 minutes ago    Up 9 minutes                                               k8s_keycloak_keycloak-6655d58b6c-rr8vq_keycloak_b95a5804-1fdc-4e3e-9dad-5c2c68fcf59b_0
69bb0fd81bc8   k8s.gcr.io/pause:3.5                  "/pause"                 9 minutes ago    Up 9 minutes                                               k8s_POD_keycloak-6655d58b6c-rr8vq_keycloak_b95a5804-1fdc-4e3e-9dad-5c2c68fcf59b_0
0e7af3260b27   6e38f40d628d                          "/storage-provisioner"   10 minutes ago   Up 10 minutes                                              k8s_storage-provisioner_storage-provisioner_kube-system_3c596e5b-edbe-4ca3-9f63-00fef2e034cd_4
e92df55c1790   k8s.gcr.io/ingress-nginx/controller   "/usr/bin/dumb-init …"   10 minutes ago   Up 10 minutes      

In [88]:
kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
keycloak-6655d58b6c-rr8vq   1/1     Running   0          9m57s


In [130]:
docker exec minikube ls /etc/kubernetes/manifests

etcd.yaml
kube-apiserver.yaml
kube-controller-manager.yaml
kube-scheduler.yaml


In [145]:
docker exec minikube cat /etc/kubernetes/manifests/kube-apiserver.yaml | grep -E  '(.*oidc.*|^)' 

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubeadm.kubernetes.io/kube-apiserver.advertise-address.endpoint: 192.168.49.2:8443
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.49.2
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
    - --kubelet-clien

In [146]:
cat ~/.kube/config

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURXekNDQWtPZ0F3SUJBZ0lJZUJ1RlVnbU5zblF3RFFZSktvWklodmNOQVFFTEJRQXdKakVrTUNJR0ExVUUKQXd3YmFXNW5jbVZ6Y3kxdmNHVnlZWFJ2Y2tBeE5qTTBNVGd6TWpNd01CNFhEVEl4TVRBeE5EQXpOVEF3TkZvWApEVEl6TVRBeE5EQXpOVEF3TlZvd0hURWJNQmtHQTFVRUF3d1NLaTVoY0hCekxXTnlZeTUwWlhOMGFXNW5NSUlCCklqQU5CZ2txaGtpRzl3MEJBUUVGQUFPQ0FROEFNSUlCQ2dLQ0FRRUEwL1ZZNm9FVERhR1RIcEkyTlZwczdHZ2IKNDJTdG1uVXphN0Ric2Q0dHBOSmpQNE5HNFE3cGNST1JOMTEvZmVFdm5Dc1orcEUwWmkxR0s3bnpBVkVYL2JMdQpZcVhkV3IrM3ArK2VaOEtDUXVWb1ozYmM5anFyNndQZUFlMzMzM1BJNXFyd1FrS2xwMm5pbVBLN095NUk0VG5CCmJ0eHVrM1l4UUJsT21OYWRDQmZkSkZEaWZqTTh3R0MxQVViVk0xN0xsbnhBV0VyejFiaXlOOVRsOXQ5Z2w3YkkKVU1iVnl3UzZackNyZmtKWEI4NU9JdnNrbWMwWHF5OHJhS2JkblBleGpWeVBzeU9vWjhHU0xyRTRmZU94dGE1UApGSXR2SkxSbmpLVk5Cemg4WEFMMlhML2FISk9UMVBDQ1hsR01YcXIyK1B0cjNrSnRQSVQ0dkMwTGlYRHZEd0lECkFRQUJvNEdWTUlHU01BNEdBMVVkRHdFQi93UUVBd0lGb0RBVEJnTlZIU1VFRERBS0JnZ3JCZ0VGQlFjREFUQU0KQmdOVkhSTUJBZjhFQWpBQU1CMEdB

In [147]:
cp ~/.kube/config ~/.kube/oidcconfig

In [89]:
kubectl config set-credentials  minikube \
   --auth-provider oidc \
   --auth-provider-arg client-id=kubernetes \
   --auth-provider-arg idp-issuer-url=https://$KEYCLOAK_HOST/auth/realms/myrealm

User "minikube" set.


In [90]:
kubectl  get pods 

Unable to connect to the server: No valid id-token, and cannot refresh without refresh-token


: 1

In [91]:
kubelogin -v6  --insecure-skip-tls-verify --user=minikube

I1112 18:36:20.794435 3545202 standalone.go:66] WARNING: log may contain your secrets such as token or password
I1112 18:36:20.795768 3545202 loader.go:372] Config loaded from file:  /home/thomas/.kube/config
NOTE: You can use the credential plugin mode for better user experience.
Kubectl automatically runs kubelogin and you do not need to run kubelogin explicitly.
See https://github.com/int128/kubelogin for more.
I1112 18:36:20.795824 3545202 standalone.go:74] using the authentication provider of the user minikube
I1112 18:36:20.795830 3545202 standalone.go:75] a token will be written to /home/thomas/.kube/config
I1112 18:36:20.795842 3545202 authentication.go:94] initializing an OpenID Connect client
I1112 18:36:20.796084 3545202 transport.go:30] GET /auth/realms/myrealm/.well-known/openid-configuration HTTP/1.1
Host: keycloak-192-168-49-2.nip.io
User-Agent: Go-http-client/1.1
Accept-Encoding: gzip

I1112 18:36:20.811150 3545202 transport.go:40] HTTP/1.1 200 OK
Content-Length: 5940
C

: 1

In [92]:
cat ~/.kube/config

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURXekNDQWtPZ0F3SUJBZ0lJZUJ1RlVnbU5zblF3RFFZSktvWklodmNOQVFFTEJRQXdKakVrTUNJR0ExVUUKQXd3YmFXNW5jbVZ6Y3kxdmNHVnlZWFJ2Y2tBeE5qTTBNVGd6TWpNd01CNFhEVEl4TVRBeE5EQXpOVEF3TkZvWApEVEl6TVRBeE5EQXpOVEF3TlZvd0hURWJNQmtHQTFVRUF3d1NLaTVoY0hCekxXTnlZeTUwWlhOMGFXNW5NSUlCCklqQU5CZ2txaGtpRzl3MEJBUUVGQUFPQ0FROEFNSUlCQ2dLQ0FRRUEwL1ZZNm9FVERhR1RIcEkyTlZwczdHZ2IKNDJTdG1uVXphN0Ric2Q0dHBOSmpQNE5HNFE3cGNST1JOMTEvZmVFdm5Dc1orcEUwWmkxR0s3bnpBVkVYL2JMdQpZcVhkV3IrM3ArK2VaOEtDUXVWb1ozYmM5anFyNndQZUFlMzMzM1BJNXFyd1FrS2xwMm5pbVBLN095NUk0VG5CCmJ0eHVrM1l4UUJsT21OYWRDQmZkSkZEaWZqTTh3R0MxQVViVk0xN0xsbnhBV0VyejFiaXlOOVRsOXQ5Z2w3YkkKVU1iVnl3UzZackNyZmtKWEI4NU9JdnNrbWMwWHF5OHJhS2JkblBleGpWeVBzeU9vWjhHU0xyRTRmZU94dGE1UApGSXR2SkxSbmpLVk5Cemg4WEFMMlhML2FISk9UMVBDQ1hsR01YcXIyK1B0cjNrSnRQSVQ0dkMwTGlYRHZEd0lECkFRQUJvNEdWTUlHU01BNEdBMVVkRHdFQi93UUVBd0lGb0RBVEJnTlZIU1VFRERBS0JnZ3JCZ0VGQlFjREFUQU0KQmdOVkhSTUJBZjhFQWpBQU1CMEdB

In [93]:
kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
keycloak-6655d58b6c-rr8vq   1/1     Running   0          11m


In [153]:
kubelogin --help

Log in to the OpenID Connect provider.

You need to set up the OIDC provider, role binding, Kubernetes API server and kubeconfig.
To show the setup instruction:

	kubectl oidc-login setup

See https://github.com/int128/kubelogin for more.

Usage:
  kubelogin [flags]
  kubelogin [command]

Available Commands:
  completion  generate the autocompletion script for the specified shell
  get-token   Run as a kubectl credential plugin
  help        Help about any command
  setup       Show the setup instruction
  version     Print the version information

Flags:
      --kubeconfig string                               Path to the kubeconfig file
      --context string                                  Name of the kubeconfig context to use
      --user string                                     Name of the kubeconfig user to use. Prior to --context
      --certificate-authority stringArray               Path to a cert file for the certificate authority
      --certificate-authority-data stringAr

In [154]:
kubelogin --insecure-skip-tls-verify

NOTE: You can use the credential plugin mode for better user experience.
Kubectl automatically runs kubelogin and you do not need to run kubelogin explicitly.
See https://github.com/int128/kubelogin for more.
Wird in einer aktuellen Browsersitzung geöffnet.
[423984:423984:0100/000000.292892:ERROR:sandbox_linux.cc(374)] InitializeSandbox() called with multiple threads in process gpu-process.
You got a valid token until 2021-11-08 19:20:51 +0100 CET


In [127]:
kubectl get pods 

NAME                        READY   STATUS    RESTARTS   AGE
keycloak-6655d58b6c-rr8vq   1/1     Running   0          46h


## alternatively use krew

Installation

```bash
(   
  set -x; 
  cd "$(mktemp -d)" &&   
  OS="$(uname | tr '[:upper:]' '[:lower:]')"  &&
  ARCH="$(uname -m | sed -e 's/x86_64/amd64/' -e 's/\(arm\)\(64\)\?.*/\1\2/' -e 's/aarch64$/arm64/')" &&
  KREW="krew-${OS}_${ARCH}" &&
  curl -fsSLO "https://github.com/kubernetes-sigs/krew/releases/latest/download/${KREW}.tar.gz" &&
  tar zxvf "${KREW}.tar.gz" &&   ./"${KREW}" install krew; 
)
```

install `oidc-login` and place it into the `$PATH`

```bash
export PATH="${KREW_ROOT:-$HOME/.krew}/bin:$PATH"
```

Update `krew` and install the `oidc-login plugin

```
kubectl krew update
kubectl krew install oidc-login

```

Now the login can be done using  `kubectl` directly

```bash
kubectl oidc-login 
``` 

with a `--insecure-skip-tls-verify` flag for test environments like ours.

In [158]:
pwd

/home/thomas/notebooks


In [95]:
curl -ks https://$KEYCLOAK_HOST/auth/realms/myrealm/protocol/openid-connect/token \
     -d grant_type=password -d client_id=kubernetes \
     -d username=myuser -d password=mypasswd \
     -d scope=openid -d response_type=id_token \
     | jq .

{
  "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJfWEFrcE8xTUVvTlFidXJwQlJrMmwyd3QzVzVtZVpyaUVnQUtERUVhOHJzIn0.eyJleHAiOjE2MzY3MzkwNzcsImlhdCI6MTYzNjczODc3NywianRpIjoiOTE4ZjlmNDgtY2I4Ny00YmQ2LTk0ZTgtMGYzZmFjMTIyYTgyIiwiaXNzIjoiaHR0cHM6Ly9rZXljbG9hay0xOTItMTY4LTQ5LTIubmlwLmlvL2F1dGgvcmVhbG1zL215cmVhbG0iLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiNDRlMjE3YTAtYzBmYy00MjRhLWI2ODUtM2MwMGI1NDU0Y2QwIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoia3ViZXJuZXRlcyIsInNlc3Npb25fc3RhdGUiOiJlMjU3ZGRiMy03MGJjLTQ1NjgtYjM1Ny05MjQwMmNlMDQwYTUiLCJhY3IiOiIxIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImRlZmF1bHQtcm9sZXMtbXlyZWFsbSIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6Im9wZW5pZCBlbWFpbCBwcm9maWxlIiwic2lkIjoiZTI1N2RkYjMtNzBiYy00NTY4LWIzNTctOTI0MDJjZTA0MGE1IiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJuYW1lIjoiTXkgVXNlciIsInByZWZlcnJlZF91c2VybmFtZSI6Im15dXNlciIsImdpdmVuX25hbWUiOiJ

In [96]:
TOKEN=$(curl -ks https://$KEYCLOAK_HOST/auth/realms/myrealm/protocol/openid-connect/token \
     -d grant_type=password -d client_id=kubernetes \
     -d username=myuser -d password=mypasswd \
     -d scope=openid -d response_type=id_token \
     | jq '.id_token' )
echo $TOKEN     
#| cut -d' ' -f 1

"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJfWEFrcE8xTUVvTlFidXJwQlJrMmwyd3QzVzVtZVpyaUVnQUtERUVhOHJzIn0.eyJleHAiOjE2MzY3MzkxMDUsImlhdCI6MTYzNjczODgwNSwiYXV0aF90aW1lIjowLCJqdGkiOiI2MGE5Y2NmZC1mMTcxLTQ2OWMtODc3Yi05N2ZmNmRkODgwNGYiLCJpc3MiOiJodHRwczovL2tleWNsb2FrLTE5Mi0xNjgtNDktMi5uaXAuaW8vYXV0aC9yZWFsbXMvbXlyZWFsbSIsImF1ZCI6Imt1YmVybmV0ZXMiLCJzdWIiOiI0NGUyMTdhMC1jMGZjLTQyNGEtYjY4NS0zYzAwYjU0NTRjZDAiLCJ0eXAiOiJJRCIsImF6cCI6Imt1YmVybmV0ZXMiLCJzZXNzaW9uX3N0YXRlIjoiNGY3ZTNhZGMtNGU2ZC00OGExLWJhYmUtODM5ZTNjNjFmZTFiIiwiYXRfaGFzaCI6Ik4wYmZ0aU45RXFCOWthNk5BY3g3QXciLCJhY3IiOiIxIiwic2lkIjoiNGY3ZTNhZGMtNGU2ZC00OGExLWJhYmUtODM5ZTNjNjFmZTFiIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJuYW1lIjoiTXkgVXNlciIsInByZWZlcnJlZF91c2VybmFtZSI6Im15dXNlciIsImdpdmVuX25hbWUiOiJNeSIsImZhbWlseV9uYW1lIjoiVXNlciJ9.V041i9W9k843fwCJUkFRtdcK-9YQYDBuZ-lXHb6s-ij4EAArkz2bm4CSNjzKnaWQU7hb7mvh_gKoDehqfW5M7M7H1wLCgF0Fr9meJeTC5b5j3rWP29zsSCAvj3sqv-XZtQfk_9becaUH2RtTAkJAl-3D-Z39L34ls5GP5hhOrT5ELIk1V3IqPxsURKUDuiS7Jhqy9QN9Xs0bq97V04

In [97]:
echo $TOKEN | sed s/\"//g| base64 --decode

{"alg":"RS256","typ" : "JWT","kid" : "_XAkpO1MEoNQburpBRk2l2wt3W5meZriEgAKDEEa8rs"}base64: ungültige Eingabe


: 1

In [98]:
cat ~/.kube/config

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURXekNDQWtPZ0F3SUJBZ0lJZUJ1RlVnbU5zblF3RFFZSktvWklodmNOQVFFTEJRQXdKakVrTUNJR0ExVUUKQXd3YmFXNW5jbVZ6Y3kxdmNHVnlZWFJ2Y2tBeE5qTTBNVGd6TWpNd01CNFhEVEl4TVRBeE5EQXpOVEF3TkZvWApEVEl6TVRBeE5EQXpOVEF3TlZvd0hURWJNQmtHQTFVRUF3d1NLaTVoY0hCekxXTnlZeTUwWlhOMGFXNW5NSUlCCklqQU5CZ2txaGtpRzl3MEJBUUVGQUFPQ0FROEFNSUlCQ2dLQ0FRRUEwL1ZZNm9FVERhR1RIcEkyTlZwczdHZ2IKNDJTdG1uVXphN0Ric2Q0dHBOSmpQNE5HNFE3cGNST1JOMTEvZmVFdm5Dc1orcEUwWmkxR0s3bnpBVkVYL2JMdQpZcVhkV3IrM3ArK2VaOEtDUXVWb1ozYmM5anFyNndQZUFlMzMzM1BJNXFyd1FrS2xwMm5pbVBLN095NUk0VG5CCmJ0eHVrM1l4UUJsT21OYWRDQmZkSkZEaWZqTTh3R0MxQVViVk0xN0xsbnhBV0VyejFiaXlOOVRsOXQ5Z2w3YkkKVU1iVnl3UzZackNyZmtKWEI4NU9JdnNrbWMwWHF5OHJhS2JkblBleGpWeVBzeU9vWjhHU0xyRTRmZU94dGE1UApGSXR2SkxSbmpLVk5Cemg4WEFMMlhML2FISk9UMVBDQ1hsR01YcXIyK1B0cjNrSnRQSVQ0dkMwTGlYRHZEd0lECkFRQUJvNEdWTUlHU01BNEdBMVVkRHdFQi93UUVBd0lGb0RBVEJnTlZIU1VFRERBS0JnZ3JCZ0VGQlFjREFUQU0KQmdOVkhSTUJBZjhFQWpBQU1CMEdB

In [99]:
kubectl plugin list

The following compatible plugins are available:

/home/thomas/bin/kubectl-foo
/home/thomas/bin/kubectl-rback
Unable to read directory "/snap/bin" from your PATH: open /snap/bin: no such file or directory. Skipping...
Unable to read directory "/usr/lib/java/bin" from your PATH: open /usr/lib/java/bin: no such file or directory. Skipping...
/home/thomas/go/bin/kubectl-oidc-login
/home/thomas/go/bin/kubectl-who-can


In [105]:
echo $USER

thomas


In [113]:
 kubectl -n ingress-nginx get deployment ingress-nginx-controller -o yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"apps/v1","kind":"Deployment","metadata":{"annotations":{},"labels":{"app.kubernetes.io/component":"controller","app.kubernetes.io/instance":"ingress-nginx","app.kubernetes.io/name":"ingress-nginx"},"name":"ingress-nginx-controller","namespace":"ingress-nginx"},"spec":{"minReadySeconds":0,"revisionHistoryLimit":10,"selector":{"matchLabels":{"app.kubernetes.io/component":"controller","app.kubernetes.io/instance":"ingress-nginx","app.kubernetes.io/name":"ingress-nginx"}},"strategy":{"rollingUpdate":{"maxUnavailable":1},"type":"RollingUpdate"},"template":{"metadata":{"labels":{"app.kubernetes.io/component":"controller","app.kubernetes.io/instance":"ingress-nginx","app.kubernetes.io/name":"ingress-nginx","gcp-auth-skip-secret":"true"}},"spec":{"containers":[{"args":["/nginx-ingress-controller","--election-id=ing

In [114]:
pwd

/home/thomas/import/training-minikube/share/notebooks/training-kubernetes-security


In [115]:
ls

AccountWithRBAC.ipynb      Linux Namespaces.ipynb             Untitled1.ipynb
assets                     Makefile                           
AuditLogs.ipynb            minikube.dot                       
auth                       minikube.png                       
Authorisation.ipynb        mkcert                             
cloudbomb                  my-namespace-rbac.dot              
ClusterOverview.ipynb      my-namespace-rbac.png              
ContainerdCloudbomb.ipynb  namespaces.dot                     
CrioBomb.ipynb             namespaces.png                     
default-rbac.png           NetworkPolicy.ipynb                
defaut-rbac.dot            OIDC-DEX.ipynb                     
docker-hypervisor.png      OIDC.ipynb                         
Harbor.ipynb               OpenPolicyAgent.ipynb              
Helm.ipynb                 OpenShift                          
Ingress.ipynb              Overview.ipynb                     
IngressKong.ipynb          PodSecurityPo

In [116]:
rm -rf rootca

In [117]:
mkdir -p certs

In [118]:
mkdir rootca

In [119]:
ls rootca

MKCert.ipynb


In [121]:
mv rootca/* .